In [1]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
import psycopg2
import calendar
import random
import seaborn as sns
import numpy as np
import itertools
import warnings
warnings.filterwarnings("ignore")

In [2]:
conn = psycopg2.connect(
  port='5432',
  host= "shopify-merchant-dump.ccm9mnr5avgs.ap-south-1.rds.amazonaws.com",
  user= "softtouchlenses_4e0b7c925f8444429457581f35678aea",
  password="softtouchlenses_4e0b7c925f8444429457581f35678aea_password",
  database= "softtouchlenses_4e0b7c925f8444429457581f35678aea_db"
)

In [3]:
query1 = """select o.id as order_id,
            o.created_at::date as order_date,
            o.customer_id as user_id,
            oli.product_id as product_id,
            oli.name as product_name,
            o.total_price as order_amount,
            CASE
        WHEN cancelled_at IS NOT NULL THEN 'Cancelled'
         ELSE 'Delivered'
     END as order_status,
            oli.quantity as quantity,
            oli.price as sp
            from orders o
            join order_line_item oli on oli.order_id = o.id"""

In [4]:
data = pd.read_sql(query1, conn)

In [5]:
data["order_date"] = pd.to_datetime(data["order_date"])
data[data["product_id"] == np.nan]
data = data.dropna()
data['WeekCount']=data['order_date'].dt.strftime('%Y-w%U')
data=data.sort_values('order_date')
WeekList=data['WeekCount'].unique()

In [6]:
WeekList = WeekList.tolist()
output_list = []

for i in range(len(WeekList) - 1):
    output_list.append(WeekList[i])
    current_week = int(WeekList[i].split('-w')[1])
    next_week = int(WeekList[i + 1].split('-w')[1])
    missing_weeks = next_week - current_week - 1
    for j in range(1, missing_weeks + 1):
        missing_week = current_week + j
        output_list.append(f'{WeekList[i][:5]}w{missing_week:02d}')

output_list.append(WeekList[-1])

In [7]:
data = data[data['order_status'] != 'Cancelled']

In [8]:
orderuserlevel=data.groupby(['order_id','user_id']).agg(
          order_date=pd.NamedAgg(column='order_date', aggfunc='min'),

    ).reset_index()
orderuserlevel=orderuserlevel.sort_values('order_date')
orderuserlevel['TrxnRank'] = orderuserlevel.groupby('user_id')['order_date'].rank(method='first')
orderuserlevel


data=pd.merge(data,
         orderuserlevel[['order_id','TrxnRank']],
         on="order_id",
         how ='left')
data['New']=0
data['Repeat']=0
data.loc[data['TrxnRank']==1,'New']=1
data.loc[data['TrxnRank']>1,'Repeat']=1

#AOV Contribution
data['ordervalue']=data['quantity']*data['sp']
data['AOVContribution']=data['ordervalue']/data["order_id"].nunique()
data['DeltaDaysinOrders']=(data.groupby('user_id').order_date.shift() - data.order_date).dt.days.abs()
data['DeltaOrderValues']=(data.groupby('user_id').ordervalue.shift() - data.ordervalue)

In [9]:
Weekly_Stats = {}
productsegstats_weekly = {}
user_cohort_labels = ['New', 'Churned (Single Use Only)', 'Recent & Loyals', 'Champions', 'Churned (Repeat Use)']
user_cohort_var_list = ['Users']#, 'Orders', 'Recency', 'AOVContribution', 'OrderValue','MeanDaysDelta', 'MeanOVDelta', 'Maturity', 'Frequency', 'AOV'
product_cohort_labels=['Hero', 'High Value & Slow Moving', 'Low Value & Slow Moving', 'New Business Drivers', 'Repeat Business Drivers']
product_cohort_var_list=['ProductCount'] #'Orders','Users','AverageSellingPrice','QuantityperCart','CartPenetration','NewUserRatio','NewOrders','ASPXQpC','AOVContribution','ProductCountMix','OrdersMix','UsersMix','AOVContribution(%)','OrderPerSKU'
final_stats = pd.DataFrame()

for x in output_list:
  x3 = data[data['WeekCount'] <= x]
  try:
    userweeklydata = x3.groupby("user_id").agg(
          Last_Date=pd.NamedAgg(column='order_date', aggfunc= lambda x: x.nlargest(2).min()),
          First_Date=pd.NamedAgg(column='order_date', aggfunc='min'),
          OrdersCount=pd.NamedAgg(column='order_id', aggfunc='nunique'),
          Frequency=pd.NamedAgg(column='order_id', aggfunc='nunique'),
          Order_Value=pd.NamedAgg(column='ordervalue', aggfunc='sum'),
          #Mean_Days_Delta=pd.NamedAgg(column='DeltaDaysinOrders', aggfunc='mean'),
          #Mean_OV_Delta=pd.NamedAgg(column='DeltaOrderValues', aggfunc='mean')
          ).reset_index()

    globalmaxdateweek=x3['order_date'].max()
    userweeklydata['Recency']=globalmaxdateweek-userweeklydata['Last_Date']
    userweeklydata['Recency']=userweeklydata['Recency'].dt.days.abs()
    userweeklydata['MaturityAge']=np.nan
    userweeklydata.loc[userweeklydata['Frequency']>1,"MaturityAge"]=(userweeklydata['Last_Date']-userweeklydata['First_Date']).dt.days.abs()
    #userweeklydata['Mean_Days_Delta']=userweeklydata['Mean_Days_Delta'].fillna(0)
    #userweeklydata['Mean_OV_Delta']=userweeklydata['Mean_OV_Delta'].fillna(0)
    userweeklydata['Customer_AOV']=userweeklydata['Order_Value']/userweeklydata['OrdersCount']

    r_labels, f_labels,AOV_labels, m_labels,DelAOV_labels = range(4, 0, -1), range(1,5), range(1,5),range(1,5),range(1,5)
    #userweeklydata['f_score'] = userweeklydata['OrdersCount'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = f_labels)).astype('int64')
    userweeklydata['r_score'] = userweeklydata['Recency'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = r_labels)).astype('int64')
   # userweeklydata['AOV_score'] = userweeklydata['Customer_AOV'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = AOV_labels)).astype('int64')
    userweeklydata['m_score']=0
    userweeklydata.loc[userweeklydata['MaturityAge'].notnull(),'m_score']= userweeklydata.loc[userweeklydata['MaturityAge'].notnull()]['MaturityAge'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = m_labels)).astype('int64')
    #userweeklydata['DelAOV'] = userweeklydata['Mean_OV_Delta'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = DelAOV_labels)).astype('int64')
    #userweeklydata['SumScore'] = userweeklydata['r_score'] + userweeklydata['m_score'] + userweeklydata['AOV_score']+ userweeklydata['DelAOV'] +userweeklydata['f_score']
    def CohortLables_3(df):
      if ((df['Frequency']==1) & (df['r_score']>2)):
          return "New"
      elif ((df['Frequency'] == 1) & (df['r_score'] <= 2)):
          return 'Churned (Single Use Only)'
      elif ((df['Frequency'] > 1) & (df['m_score'] < 3) & (df['r_score'] >2)):
          return 'Recent & Loyals'
      elif ((df['Frequency'] > 1) & (df['m_score'] >=3) & (df['r_score'] > 2)):
          return 'Champions'
      else:
          return 'Churned (Repeat Use)'
    userweeklydata['User_Cohort_Label'] = userweeklydata.apply(CohortLables_3, axis=1)
    x3=pd.merge(x3,
         userweeklydata[['user_id','User_Cohort_Label']],
         on="user_id",
         how ='left')

    totalcarts=x3['order_id'].nunique()
    productlevel=x3.groupby(['product_id','product_name']).agg(
              Carts=pd.NamedAgg(column='order_id', aggfunc='nunique'),
              quantity=pd.NamedAgg(column='quantity', aggfunc='sum'),
              ASP=pd.NamedAgg(column='sp', aggfunc='mean'),
              NewOrders=pd.NamedAgg(column='New', aggfunc='sum'),
              RepeatOrders=pd.NamedAgg(column='Repeat', aggfunc='sum'),
              Users=pd.NamedAgg(column='user_id', aggfunc='nunique'),
              AOVContribution=pd.NamedAgg(column='AOVContribution', aggfunc='sum'),
        ).reset_index()
    productlevel['CartPenetration']=productlevel['Carts']/totalcarts
    productlevel['NewUserRatio']=productlevel['NewOrders']/productlevel['Users']
    productlevel['QpC']=productlevel['quantity']/productlevel['Carts']
    productlevel['ASPXQpC']=productlevel['ASP']*productlevel['QpC']

    CP_labels, NewRatio_labels,ASP_labels, QpC_labels =  range(1,5), range(1,5),range(1,5),range(1,5)

    productlevel['CP_score'] = productlevel['CartPenetration'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = CP_labels)).astype('int64')
    productlevel['NewRatio_score'] = productlevel['NewUserRatio'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = NewRatio_labels)).astype('int64')
    productlevel['ASP_score'] = productlevel['ASP'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = ASP_labels)).astype('int64')
    productlevel['QpC_score'] = productlevel['QpC'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = QpC_labels)).astype('int64')
    productlevel['SumScore'] = productlevel['CP_score'] + productlevel['NewRatio_score'] + productlevel['ASP_score'] + productlevel['QpC_score']

    def ProductCohortLables2(df):
      if ((df['CP_score']>2) and (df['NewRatio_score'] >3)):
          return 'New Business Drivers'
      elif ((df['CP_score'] >2) and (df['NewRatio_score'] <2)):
          return 'Repeat Business Drivers'
      elif (df['CP_score'] >2):
          return 'Hero'
      elif (df['ASP_score'] >3):
          return 'High Value & Slow Moving'
      elif (df['ASP_score'] <=3):
          return 'Low Value & Slow Moving'
      else:
          return 'Low Value & Slow Moving'

    productlevel['Product Cohort Lables'] = productlevel.apply(ProductCohortLables2, axis=1)
    x3=pd.merge(x3,
         productlevel[['product_id','Product Cohort Lables']],
         on="product_id",
         how ='left')


    User_Product_Level=x3.groupby(['User_Cohort_Label','Product Cohort Lables']).agg(
        Order_Value=pd.NamedAgg(column='ordervalue', aggfunc='sum'),
            Users=pd.NamedAgg(column='user_id', aggfunc='nunique'),
        Order_Count=pd.NamedAgg(column='order_id', aggfunc='nunique'),

    ).reset_index()

    Weekly_Stats[x] = User_Product_Level
    x4 = data[data['WeekCount']==x]
    x5 = pd.merge(x4,userweeklydata[['user_id','User_Cohort_Label']], on='user_id', how='left')
    x5 = pd.merge(x5,productlevel[['product_id','Product Cohort Lables']], on='product_id', how='left')
    final_stats = final_stats.append(x5, ignore_index=True)
  except:
    pass


In [10]:
#AOV Contribution
final_stats['ordervalue']=final_stats['ordervalue']
final_stats['AOVContribution']=final_stats['ordervalue']/final_stats["order_id"].nunique()
final_stats['DeltaDaysinOrders']=(final_stats.groupby('user_id').order_date.shift() - final_stats.order_date).dt.days.abs()
final_stats['DeltaOrderValues']=(final_stats.groupby('user_id').ordervalue.shift() - final_stats.ordervalue)
weekly_summary_stats = final_stats.groupby(['WeekCount','User_Cohort_Label','Product Cohort Lables']).agg(
            totalorders=pd.NamedAgg(column='order_id', aggfunc='nunique'),
            totalusers=pd.NamedAgg(column='user_id', aggfunc='nunique'),
            totalvalue=pd.NamedAgg(column='ordervalue', aggfunc='sum'),
            quantity=pd.NamedAgg(column='quantity', aggfunc='sum'),
            SKUCounts=pd.NamedAgg(column='product_id', aggfunc='nunique')

      ).reset_index()
newdf=final_stats[final_stats['New']==1].groupby(['WeekCount','User_Cohort_Label','Product Cohort Lables']).agg(
    newusercount=pd.NamedAgg(column='user_id', aggfunc='nunique'),
     ).reset_index()
cancelled_orders = final_stats[final_stats['order_status'] == 'Cancelled'].groupby(['WeekCount','User_Cohort_Label','Product Cohort Lables']).agg(
    CancellationCount=pd.NamedAgg(column='order_id', aggfunc='count')
).fillna(0).reset_index()

delivered_orders = final_stats[final_stats['order_status'] == 'Delivered'].groupby(['WeekCount','User_Cohort_Label','Product Cohort Lables']).agg(
    DeliveryCount=pd.NamedAgg(column='order_id', aggfunc='count')
).fillna(0).reset_index()
weekly_summary_stats = weekly_summary_stats.merge(newdf, on=['WeekCount','User_Cohort_Label','Product Cohort Lables'], how="left")
weekly_summary_stats =weekly_summary_stats.merge(cancelled_orders, on=['WeekCount','User_Cohort_Label','Product Cohort Lables'], how="left")
weekly_summary_stats = weekly_summary_stats.merge(delivered_orders, on=['WeekCount','User_Cohort_Label','Product Cohort Lables'], how="left")
weekly_summary_stats['repeatusercount']=weekly_summary_stats['totalusers']-weekly_summary_stats['newusercount']
weekly_summary_stats['AOV'] = weekly_summary_stats['totalvalue']/ weekly_summary_stats['totalorders']

In [11]:
weekly_summary_stats

,WeekCount,User_Cohort_Label,Product Cohort Lables,totalorders,totalusers,totalvalue,quantity,SKUCounts,newusercount,CancellationCount,DeliveryCount,repeatusercount,AOV
0,2021-w33,Churned (Single Use Only),Low Value & Slow Moving,1,1,599.0,1,1,1.0,NaN,1,0.0,599.000000
1,2021-w33,Churned (Single Use Only),New Business Drivers,1,1,899.0,1,1,1.0,NaN,1,0.0,899.000000
2,2021-w33,New,Hero,1,1,900.0,2,1,1.0,NaN,1,0.0,900.000000
3,2021-w33,New,High Value & Slow Moving,1,1,1399.0,1,1,1.0,NaN,1,0.0,1399.000000
4,2021-w33,New,Low Value & Slow Moving,1,1,600.0,1,1,1.0,NaN,1,0.0,600.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1040,2023-w50,New,Low Value & Slow Moving,12,12,41091.0,46,7,12.0,NaN,46,0.0,3424.250000
1041,2023-w50,New,New Business Drivers,7,7,19178.0,18,4,7.0,NaN,18,0.0,2739.714286
1042,2023-w50,New,Repeat Business Drivers,18,18,263552.0,194,11,18.0,NaN,193,0.0,14641.777778
1043,2023-w50,Recent & Loyals,High Value & Slow Moving,1,1,38320.0,16,1,NaN,NaN,4,NaN,38320.000000


In [12]:
final_stats["Retention"] = 0
final_stats.loc[(final_stats["DeltaDaysinOrders"] > 0) & (final_stats["DeltaDaysinOrders"] <= 90) , "Retention"] = "Retained"
final_stats.loc[(final_stats["DeltaDaysinOrders"] > 90), "Retention"] = "Not-Retained"
retained_df = final_stats[final_stats['Retention'] == 'Retained'].groupby(['WeekCount','User_Cohort_Label','Product Cohort Lables']).size().reset_index(name='Retained_Users')
weekly_summary_stats = pd.merge(weekly_summary_stats, retained_df, on=['WeekCount','User_Cohort_Label','Product Cohort Lables'], how="left")

In [13]:
key_metrices = ['WeekCount','User_Cohort_Label','Product Cohort Lables','totalorders', 'totalvalue', 'repeatusercount','newusercount', 'AOV']
weekly_summary_stats = weekly_summary_stats[key_metrices]

weekly_summary_stats = weekly_summary_stats.rename(columns={
    'totalorders':'Total_Orders',
    'totalvalue': 'Total_Value',
    'repeatusercount': 'No_of_Repeat_Users',
    'newusercount': 'No_of_New_Users(First_Time_Users)',
    'AOV': 'Average_Order_Value'
})
weekly_summary_stats=weekly_summary_stats.fillna(0)

In [14]:
weekly_summary_stats['AOV_Contribution'] = weekly_summary_stats['Total_Value'] / weekly_summary_stats.groupby('WeekCount')['Total_Orders'].transform('sum')

Component 1

In [15]:
grouped_by_week = weekly_summary_stats.groupby('WeekCount')
dataframes_by_week = {}
for week, group in grouped_by_week:
    dataframes_by_week[week] = group.drop(columns='WeekCount').copy()

In [16]:
combinations = list(itertools.product(user_cohort_labels, product_cohort_labels))
combinations_df_by_week = pd.DataFrame(combinations, columns=['User_Cohort_Label', 'Product Cohort Lables'])

In [17]:
for df_name, df in dataframes_by_week.items():
    merged_df = pd.merge(
        combinations_df_by_week,
        df,
        how='outer',
        on=['User_Cohort_Label', 'Product Cohort Lables']
    )
    merged_df.fillna(0, inplace=True)
    dataframes_by_week[df_name] = merged_df


In [18]:
def week_metric_user_product_split(week, parameter):
  week_df = dataframes_by_week[week]
  pivot_df = week_df.pivot_table(index='User_Cohort_Label', columns='Product Cohort Lables', values=parameter, aggfunc='sum')
  return pivot_df
df_output_Week = week_metric_user_product_split("2023-w44", "Total_Value")

In [19]:
df_output_Week

Product Cohort Lables,Hero,High Value & Slow Moving,Low Value & Slow Moving,New Business Drivers,Repeat Business Drivers
User_Cohort_Label,,,,,
Champions,74668.0,156240.0,9100.0,210621.0,273989.0
Churned (Repeat Use),39000.0,50832.0,23600.0,77148.0,96692.0
Churned (Single Use Only),0.0,0.0,0.0,0.0,0.0
New,260243.0,129423.0,39183.0,33727.0,168903.0
Recent & Loyals,24784.0,9858.0,32062.0,998.0,3797.0


Component 2

In [20]:
weekly_summary_stats["key"] = weekly_summary_stats["User_Cohort_Label"] + "-" + weekly_summary_stats["Product Cohort Lables"]
weekly_summary_stats.drop(['User_Cohort_Label', 'Product Cohort Lables'], axis=1, inplace=True)
weekly_summary_stats.set_index("WeekCount", inplace=True)

In [21]:
grouped_dict = {week: group for week, group in weekly_summary_stats.groupby(level='WeekCount')}
for week, df in grouped_dict.items():
    df = df.reset_index(drop=True)
    df.set_index('key', inplace=True)
    grouped_dict[week] = df

In [22]:
all_combinations = list(itertools.product(user_cohort_labels, product_cohort_labels))
formatted_combinations = [f'{user}-{product}' for user, product in all_combinations]

In [23]:
for key, df in grouped_dict.items():
    missing_indices = [index for index in formatted_combinations if index not in df.index]
    if missing_indices:
        for missing_index in missing_indices:
            df.loc[missing_index] = 0

In [24]:
averages_dict = {}
weeks = list(grouped_dict.keys())

for i in range(8, len(weeks)):
    week = weeks[i]
    df = grouped_dict[week]
    avg_df = df.rolling(window=8, min_periods=1).mean().shift(1)
    avg_df = avg_df.iloc[1:]
    averages_dict[week] = avg_df

In [25]:
for key, df in averages_dict.items():
    missing_indices = [index for index in formatted_combinations if index not in df.index]
    if missing_indices:
        for missing_index in missing_indices:
            df.loc[missing_index] = 0

In [26]:
result_list = []

for key, dataframe in grouped_dict.items():
    sum_result = pd.DataFrame(dataframe.sum(), columns=['sum_result']).T
    sum_result['key'] = key
    sum_result['original_index'] = dataframe.index[0]
    sum_result.set_index('original_index', inplace=True)
    result_list.append(sum_result)

result_df_1= pd.concat(result_list, ignore_index=True)

In [27]:
result_df_melted_1 = pd.melt(result_df_1, id_vars=['key'], var_name='Metric', value_name='Value')
result_df_melted_1.rename(columns={'key': 'WeekCount'}, inplace=True)

In [28]:
result_list = []

for key, dataframe in averages_dict.items():
    sum_result = pd.DataFrame(dataframe.sum(), columns=['sum_result']).T  # Convert sum_result to a DataFrame and transpose
    sum_result['key'] = key
    sum_result['original_index'] = dataframe.index[0]  # Use the first index value
    sum_result.set_index('original_index', inplace=True)
    result_list.append(sum_result)
result_df_2= pd.concat(result_list, ignore_index=True)

In [29]:
result_df_melted_2 = pd.melt(result_df_2, id_vars=['key'], var_name='Metric', value_name='Value')
result_df_melted_2.rename(columns={'key': 'WeekCount'}, inplace=True)

In [30]:
def create_metric_df(data_dict, metric_column):
    curr_week_df = pd.DataFrame()

    for week, dataframe in data_dict.items():
        current_index_list = dataframe.index.tolist()
        current_metric_values = dataframe[metric_column].tolist()

        temp_df = pd.DataFrame({
            'Metric': current_index_list,
            'Value': current_metric_values,
            'WeekCount': [week] * len(current_index_list)
        })
        curr_week_df = curr_week_df.append(temp_df, ignore_index=True)

    curr_week_df = curr_week_df.rename(columns={
        'Metric': 'Metric',
        'Value': 'Value',
        'WeekCount': 'WeekCount'
    })

    return curr_week_df
curr_week_df = create_metric_df(grouped_dict, 'Total_Value')

In [31]:
def create_average_df(data_dict, metric_column):

    avg_week_df = pd.DataFrame()

    for week, dataframe in data_dict.items():
        current_index_list = dataframe.index.tolist()
        current_metric_values = dataframe[metric_column].tolist()

        temp_df = pd.DataFrame({
            'Metric': current_index_list,
            'Value': current_metric_values,
            'WeekCount': [week] * len(current_index_list)
        })
        avg_week_df = avg_week_df.append(temp_df, ignore_index=True)

    avg_week_df = avg_week_df.rename(columns={
        'Metric': 'Metric',
        'Value': 'Value',
        'WeekCount': 'WeekCount'
    })
    return avg_week_df

avg_week_df = create_average_df(averages_dict, 'Total_Value')


In [32]:
current_week_dataframe = pd.concat([result_df_melted_1, curr_week_df], axis=0)
current_week_dataframe = current_week_dataframe.sort_values(by='WeekCount', ascending=True)
current_week_dataframe.rename(columns={'Metric': 'Variables', 'Value' : 'Current Week Value'}, inplace=True)
keys_list = list(averages_dict.keys())
current_week_dataframe = current_week_dataframe[current_week_dataframe['WeekCount'].isin(keys_list)]

In [33]:
last_8_week_dataframe = pd.concat([result_df_melted_2, avg_week_df], axis=0)
last_8_week_dataframe = last_8_week_dataframe.sort_values(by='WeekCount', ascending=True)
last_8_week_dataframe.rename(columns={'Metric': 'Variables', 'Value' : 'Last 8 Week Value'}, inplace=True)

In [34]:
merged_dataframe = pd.merge(current_week_dataframe, last_8_week_dataframe, on=['WeekCount', 'Variables'], how='left')
merged_dataframe = merged_dataframe.fillna(0)
primary_explanatory_vars = ['Total_Orders', 'No_of_Repeat_Users', 'AOV_Contribution', 'No_of_New_Users(First_Time_Users)', '90_Day_Retention']
target_vars = ["Total_Value"]
merged_dataframe.loc[merged_dataframe['Variables'].isin(target_vars), 'Tag'] = 'Target_Variable'
merged_dataframe.loc[merged_dataframe['Variables'].isin(primary_explanatory_vars), 'Tag'] = 'Primary_Explanatory'
merged_dataframe.loc[merged_dataframe['Variables'].isin(formatted_combinations), 'Tag'] = 'Secondary_Explanatory'
merged_dataframe['Difference'] = merged_dataframe['Current Week Value'] - merged_dataframe['Last 8 Week Value']

In [35]:
new_df_1 = merged_dataframe[merged_dataframe["Tag"] == "Secondary_Explanatory"]
new_df_1

,WeekCount,Variables,Current Week Value,Last 8 Week Value,Tag,Difference
0,2021-w42,Churned (Single Use Only)-New Business Drivers,0.0,638.250,Secondary_Explanatory,-638.250
1,2021-w42,Churned (Single Use Only)-Low Value & Slow Moving,0.0,813.125,Secondary_Explanatory,-813.125
2,2021-w42,Churned (Single Use Only)-High Value & Slow Mo...,0.0,1337.750,Secondary_Explanatory,-1337.750
3,2021-w42,Churned (Single Use Only)-Hero,0.0,1512.625,Secondary_Explanatory,-1512.625
4,2021-w42,Churned (Single Use Only)-Repeat Business Drivers,0.0,399.750,Secondary_Explanatory,-399.750
...,...,...,...,...,...,...
3492,2023-w50,Champions-High Value & Slow Moving,9294.0,59420.000,Secondary_Explanatory,-50126.000
3493,2023-w50,Champions-Hero,59420.0,0.000,Secondary_Explanatory,59420.000
3499,2023-w50,Churned (Repeat Use)-New Business Drivers,0.0,0.000,Secondary_Explanatory,0.000
3501,2023-w50,New-New Business Drivers,19178.0,35813.375,Secondary_Explanatory,-16635.375


In [36]:
new_df_1[['User_Cohort_Lables', 'Product_Cohort_Lables']] = new_df_1['Variables'].str.split('-', expand=True)

In [37]:
grouped_by_week_delta = new_df_1.groupby('WeekCount')
dataframes_by_week = {}
for week, group in grouped_by_week_delta:
    dataframes_by_week[week] = group.drop(columns='WeekCount').copy()

In [38]:
def week_metric_user_product_split(week, parameter):
  week_df = dataframes_by_week[week]
  pivot_df = week_df.pivot_table(index='User_Cohort_Lables', columns='Product_Cohort_Lables', values=parameter, aggfunc='sum')
  return pivot_df
df_output_delta = week_metric_user_product_split("2023-w44", "Difference")

In [43]:
df_output_Week

Product Cohort Lables,Hero,High Value & Slow Moving,Low Value & Slow Moving,New Business Drivers,Repeat Business Drivers
User_Cohort_Label,,,,,
Champions,74668.0,156240.0,9100.0,210621.0,273989.0
Churned (Repeat Use),39000.0,50832.0,23600.0,77148.0,96692.0
Churned (Single Use Only),0.0,0.0,0.0,0.0,0.0
New,260243.0,129423.0,39183.0,33727.0,168903.0
Recent & Loyals,24784.0,9858.0,32062.0,998.0,3797.0


In [39]:
df_output_delta

Product_Cohort_Lables,Hero,High Value & Slow Moving,Low Value & Slow Moving,New Business Drivers,Repeat Business Drivers
User_Cohort_Lables,,,,,
Champions,74668.000,81572.000000,-106354.000,130618.333333,161331.750
Churned (Repeat Use),-105923.600,-76437.666667,-92750.000,-27608.250000,-8374.250
Churned (Single Use Only),-39164.000,-34266.125000,-30050.250,-8937.375000,-5839.375
New,162620.250,407.375000,-79682.875,-55788.125000,80047.000
Recent & Loyals,-78830.875,-93904.875000,-63289.625,-86274.875000,-51070.250


In [40]:
df_output_Week.sum().sum()

1714868.0

In [41]:
df_output_delta.sum().sum()

-353281.68333333335

In [42]:
delta_df = merged_dataframe[merged_dataframe["WeekCount"] == "2023-w44"]
selected_row = delta_df.loc[(delta_df['WeekCount'] == "2023-w44") & (delta_df['Variables'] == "Total_Value")]
selected_row

,WeekCount,Variables,Current Week Value,Last 8 Week Value,Tag,Difference
3311,2023-w44,Total_Value,1714868.0,2.068150e+06,Target_Variable,-353281.683333
